In [1]:
%load_ext autoreload
%autoreload 2

import re
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 12, 6
matplotlib.rcParams['font.size'] = 18  # Probably OS Dependent

In [3]:
data_files = glob('../public/*.json')

dfs = []
for file_ in data_files:
    df = pd.read_json(file_)
    df['state'] = re.search('public/(.+).json', file_).group(1).title()
    dfs += [df]
    
df = pd.concat(dfs).reset_index()
df.sample(10, random_state=422)

,index,locale,official,emails,faxes,phones,county,address,party,state,contacts,physicalAddress,cityType,city,url,faxs,urls
1357,1105,Town of Three Lakes:Oneida County,Susan L Harris,[TOWNCLERKSUE@TOWNOFTHREELAKES.COM],[(715)546-3384],NaN,Oneida County,"Po Box 565, Three Lakes, Wi 54562-0565",NaN,Wisconsin,NaN,"6965 W School St, Three Lakes, Wi 54562",Town,Three Lakes,NaN,NaN,NaN
1681,1429,Village of Radisson:Sawyer County,Gwen Genari,[vclerk@bevcomm.net],[],NaN,Sawyer County,"Po Box 127, Radisson, Wi 54867-0127",NaN,Wisconsin,NaN,"10598 W Railroad St, Radisson, Wi 54867",Village,Radisson,NaN,NaN,NaN
2068,1816,City of Neenah:Winnebago County,Patty Sturn,"[PSTURN@CI.NEENAH.WI.US, scheslock@ci.neenah.w...",[(920)886-6109],NaN,Winnebago County,"Po Box 426, Neenah, Wi 54956-0426",NaN,Wisconsin,NaN,"211 Walnut St, Neenah, Wi 54956-3026",City,Neenah,NaN,NaN,NaN
3444,678,Marion Township:Sanilac County,Deborah G Williamson,[None],[],[(810) 376-8229],Sanilac County,NaN,NaN,Michigan,NaN,NaN,NaN,Marion Township,NaN,NaN,NaN
4361,60,St. Lucie County,Gertrude Walker,[elections@slcelections.com],NaN,NaN,St. Lucie County,NaN,NaN,Florida,NaN,NaN,NaN,NaN,http://www.slcelections.com,NaN,NaN
2976,210,Lincoln Township:Huron County,Irvin Kanaski,[ikanaski1903@yahoo.com],[],[(989) 874-5055],Huron County,NaN,NaN,Michigan,NaN,NaN,NaN,Lincoln Township,NaN,NaN,NaN
2999,233,Gaastra City:IRON COUNTY,Debra Tusa,[gaastrac@fast-air.net],[(906) 265-2440],[(906) 265-2141],IRON COUNTY,NaN,NaN,Michigan,NaN,NaN,NaN,Gaastra City,NaN,NaN,NaN
925,673,Village of Ridgeway:Iowa County,Hailey Roessler,[clerk@villageofridgeway.com],[(888)275-8057],NaN,Iowa County,"113 Dougherty Ct, Ridgeway, Wi 53582-9778",NaN,Wisconsin,NaN,"113 Dougherty Ct, Ridgeway, Wi 53582-9778",Village,Ridgeway,NaN,NaN,NaN
212,119,Gwinnett County,Kristi L. Royston,[voterregistration@gwinnettcounty.com],[(678) 226-7208],NaN,Gwinnett County,NaN,NaN,Georgia,{'Mailing Address': {'kind': 'Mailing Address'...,NaN,NaN,NaN,NaN,NaN,NaN
3612,846,Norvell Township:Jackson County,Gloria Hubbell,[ghubbell@norvelltownship.com],[(517) 536-0110],[(517) 536-4370],Jackson County,NaN,NaN,Michigan,NaN,NaN,NaN,Norvell Township,NaN,NaN,NaN


In [4]:
def non_empty_element(el_list):
    if type(np.nan) == float:
        return False
    values = [len(x.strip()) > 0 for x in el_list if x]
    if len(values) == 0:
        return False
    return any(values)

def not_null_empty(series):
    return (series.notnull() & series.apply(lambda x: x != ''))

email = df['emails'].apply(non_empty_element)
fax = df['faxes'].apply(non_empty_element)
city = not_null_empty(df['city'])
county = not_null_empty(df['county'])

df_valid = pd.DataFrame({
    'locale': not_null_empty(df['locale']),
    'city': city,
    'county': county,
    'city_county': (city | county),
    'official': df['official'].notnull(),
    'email': email,
    'faxes': fax,
    'fax_email': (email | fax),
})


df_stat = df_valid.groupby(df['state']).mean()
df_stat['records'] = df.groupby('state').size()
df_stat

,locale,city,county,city_county,official,email,faxes,fax_email,records
state,,,,,,,,,
Florida,1.0,0.000000,1.000000,1.0,1.0,0.0,0.0,0.0,67
Georgia,1.0,0.000000,1.000000,1.0,1.0,0.0,0.0,0.0,159
Maine,1.0,1.000000,0.000000,1.0,1.0,0.0,0.0,0.0,504
Maryland,1.0,0.000000,1.000000,1.0,1.0,0.0,0.0,0.0,24
Michigan,1.0,1.000000,1.000000,1.0,1.0,0.0,0.0,0.0,1535
Minnesota,1.0,0.000000,1.000000,1.0,1.0,0.0,0.0,0.0,87
Nebraska,1.0,0.000000,1.000000,1.0,1.0,0.0,0.0,0.0,93
Virginia,1.0,0.285714,0.714286,1.0,1.0,0.0,0.0,0.0,133
Wisconsin,1.0,1.000000,1.000000,1.0,1.0,0.0,0.0,0.0,1853
